In [2]:
from datasets import load_dataset

ds = load_dataset('ganga4364/garchen_rinpoche_data_20hours')
ds

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.08M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/547k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17739 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3131 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file_name', 'uni', 'wylie', 'url', 'dept', 'grade', 'char_len', 'audio_len', '__index_level_0__'],
        num_rows: 17739
    })
    validation: Dataset({
        features: ['file_name', 'uni', 'wylie', 'url', 'dept', 'grade', 'char_len', 'audio_len', '__index_level_0__'],
        num_rows: 3131
    })
})

In [3]:
import librosa
import requests
import tempfile

def prepare_dataset(batch):
    # Download audio from URL
    response = requests.get(batch["url"])
    response.raise_for_status()

    # Save to temporary WAV file
    with tempfile.NamedTemporaryFile(suffix=".wav") as tmp:
        tmp.write(response.content)
        tmp.flush()

        # Load and resample audio using librosa
        waveform, sr = librosa.load(tmp.name, sr=16000)
        batch["audio"] = {"array": waveform, "sampling_rate": sr}
        batch["transcript"] = batch["uni"]

    return batch

ds = ds.map(prepare_dataset, num_proc=1)

Map (num_proc=1):   0%|          | 0/17739 [00:00<?, ? examples/s]

Map (num_proc=1):   0%|          | 0/3131 [00:00<?, ? examples/s]

In [4]:
ds.save_to_disk('garchen_dataset')

Saving the dataset (0/10 shards):   0%|          | 0/17739 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/3131 [00:00<?, ? examples/s]